# 🌦️Framstiling av værdataanalyse

I denne notebooken finner du er visuell representasjon av dataen vi har innsamlet, sammen med vurderinger og konklusjoner rundt funnene vi har gjort

Innholdsfortegnelse:

[DEl 1 - Datainnsamling](del-1---datainnsamling)

---
### 📊Del 1 - Datainnsamling

Denne delen består at tre funskjoner som vi har brukt for å hente data fra Frost og Kaggle, den første koden henter fra Frost, her har vi hentet data ved hjelp av en API-nøkkel. Det siste datasetett er lastet ned lokalt som en CSV, men vi ønsket å gjøre den om til en JSON fil slik at det ble lettere for oss å lese og håndere filen.

For å se kodene bak funksjonene ligger de i 

🔗 [FrostBlindern.py](./FrostBlindern.py)
🔗 [KaggleLondon.py](./KaggleLondon.py)
🔗 [KaggleBoston.py](./KaggleBoston.py)



In [5]:
import os
from dotenv import load_dotenv

#Henter og lagere værdata fra Blindern

# Last .env fra src-mappen (der notebook og .env ligger)
load_dotenv(dotenv_path=os.path.join(os.getcwd(), '.env'))

import sys
sys.path.append(os.getcwd())
from FrostBlindern import hent_og_lagre_data

# Patch exit for notebook
import builtins
builtins.exit = lambda code=0: (_ for _ in ()).throw(SystemExit(code))

# Kjør funksjonen uten å vise returverdien
_= hent_og_lagre_data()

API-nøkkel funnet og data sett lastet inn!
Data saved to 'c:\anvendt_prog\Anvendt_prosjekt\data\observations_data.json'


In [6]:
# Henter og lagrer værdata fra Boston i en CSV
import sys
import os

# Sørg for at data-mappen utenfor src eksisterer før du lagrer CSV
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(project_root, 'data')
os.makedirs(data_dir, exist_ok=True)

sys.path.append(os.getcwd())
from KaggleBoston import hent_boston_data_csv

# Filnavn skal kun være filnavn, ikke path, siden funksjonen lager path selv
hent_boston_data_csv("BostonData2.csv", start_aar=2013, slutt_aar=2023)

Boston-data er hentet og lagret i data-mappen!


In [7]:
import sys
import os

# Bruk korrekt relativ sti til data-mappen utenfor src
csv_path = os.path.abspath(os.path.join("..", "data", "london_weather.csv"))
json_path = os.path.abspath(os.path.join("..", "data", "london_weather.json"))

if not os.path.exists(csv_path):
    print(f"Filen {csv_path} finnes ikke! Sjekk at du har lastet ned eller generert riktig CSV.")
else:
    sys.path.append(os.getcwd())
    from KaggleLondon import csv_til_json_london
    csv_til_json_london(csv_path, json_path)

Konvertert til JSON og lagret som c:\anvendt_prog\Anvendt_prosjekt\data\london_weather.json


---
### Del 2 - Dataforbredelse